<a href="https://colab.research.google.com/github/vyom10445/toxicity-classifier/blob/main/notebooks/Toxicity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [10]:
import os
os.chdir("toxicity-classifier")

In [11]:
df = pd.read_csv(
    os.path.join(
        "data",
        "jigsaw-toxic-comment-classification-challenge",
        "train.csv"
    )
)

In [14]:
from tensorflow.keras.layers import TextVectorization #preprocessing

In [15]:
x = df['comment_text']
y = df[df.columns[2:]].values

In [16]:
MAX_FEATURES = 200000 # number of words in the vocab

In [17]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length=1800,
                               output_mode='int')

In [19]:
vectorizer.adapt(x.values)

In [21]:
vectorized_text = vectorizer(x.values)

In [22]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) # helps prevent bottlenecks

In [23]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))